# Hello World Example

This example demonstrates how to use the Dioptra Python client to register the necessary resources in a Dioptra deployment to execute a simple job.

It assumes you have an existing Dioptra deployment.

### Configure the Dioptra client and authenticate

The first cell below imports the functions we will be using from the Dioptra client, instantiates the client, and authenticates a user.

Edit the REST API address and username/password as needed.

If successful, you should see a "Login successful" status message.

In [ ]:
from pathlib import Path
from pprint import pprint
from dioptra.client import connect_json_dioptra_client, select_files_in_directory

DIOPTRA_REST_API_ADDRESS = "http://localhost:5000"
DIOPTRA_REST_API_USER = "<user>"
DIOPTRA_REST_API_PASS = "<password>"

client = connect_json_dioptra_client(DIOPTRA_REST_API_ADDRESS)

# if you have not yet registered a user, uncomment the following line and adjust the parameters as desired to register a user first
# client.users.create(DIOPTRA_REST_API_USER, email=f"{DIOPTRA_REST_API_USER}@localhost", password=DIOPTRA_REST_API_PASS)

client.auth.login(DIOPTRA_REST_API_USER, DIOPTRA_REST_API_PASS)

### Import Resources

We will import the resources (plugins, entrypoints, and plugin parameter types) used in this example.
It is possible to import from a git repository or from files on the local filesystem. Choose one of the two options below.

If successful, you should see a message listing the imported resources. If you have resouces with a conflicting name, you will receive an error. You can either rename your existing resources or overwrite on import by passing `resolve_name_conflicts_strategy="overwrite"` below.

In [ ]:
# import from git
response = client.workflows.import_resources(
    group_id=1,
    source="https://github.com/usnistgov/dioptra.git;extra#dev",
    config_path="extra/dioptra.toml",
    # resolve_name_conflicts_strategy="overwrite",
)
resources = response["resources"]
ENTRYPOINT_IDS = resources["entrypoints"]
pprint(resources)

In [ ]:
# import from local filesystem
response = client.workflows.import_resources(
    group_id=1,
    source=select_files_in_directory("../extra/", recursive=True),
    resolve_name_conflicts_strategy="overwrite",
)
resources = response["resources"]
ENTRYPOINT_IDS = resources["entrypoints"]
pprint(resources)

### Configure Experiment and Workers

We need to create an experiment to act as a namespace to organize our jobs.

We also to create a queue to submit jobs to. The queue name must match the name of a running worker in your deployment. Edit the variable below if necessary.

Finally, we associate the experiment with the entrypoint we wish to run and make the entrypoint submissible to our queue.

In [ ]:
EXPERIMENT_NAME = "Hello World"
QUEUE_NAME = "Demo Worker"

try:
    experiment = client.experiments.create(group_id=1, name=EXPERIMENT_NAME)
except:
    experiment = client.experiments.get(search=f"name:'{EXPERIMENT_NAME}'")["data"][0]

try:
    queue = client.queues.create(group_id=1, name=QUEUE_NAME)
except:
    queue = client.queues.get(search=f"name:'{QUEUE_NAME}'")["data"][0]

client.experiments.entrypoints.create(
    experiment_id=experiment["id"], entrypoint_ids=list(ENTRYPOINT_IDS.values())
)
client.entrypoints.queues.create(
    entrypoint_id=ENTRYPOINT_IDS["Hello World"], queue_ids=[queue["id"]]
)
client.entrypoints.queues.create(
    entrypoint_id=ENTRYPOINT_IDS["Hello Artifact"], queue_ids=[queue["id"]]
)

print("Successfully created experiment and queue, and associated resources.")

### Execute the job

Now that everything is registered in Dioptra, we can run a job. Run the following cell to execute the job. You should see messages printed in the logs of the worker. You can also check the status of the job by running the subsequent cell. You should see it move from `queued` to `started` to `finished` over the course of a couple seconds. Try changing the value of the name variable.

In [ ]:
job1_response = client.experiments.jobs.create(
    experiment_id=experiment["id"],
    queue_id=queue["id"],
    entrypoint_id=resources["entrypoints"]["Hello World"],
    values={"name": DIOPTRA_REST_API_USER},
)

Check on the status of the job until it is `finished`.

In [ ]:
job1_info = client.jobs.get_by_id(job_id=job1_response["id"])
pprint(job1_info)

Once the Job is completed, we can download the artifact it produced, read in the file, and print it to the screen.

In [ ]:
artifact_info = client.artifacts.get_by_id(job1_info["artifacts"][0]["id"])
filename = Path(artifact_info["artifactUri"]).name
client.artifacts.get_contents(artifact_info["id"], file_stem=filename)
print(Path(filename).read_text())

Run a job for the second endpoint, which uses the artifact produced by the first job as an input to a plugin task.

In [ ]:
job2_response = client.experiments.jobs.create(
    experiment_id=experiment["id"],
    queue_id=queue["id"],
    entrypoint_id=resources["entrypoints"]["Hello Artifact"],
    artifact_values={
        "greeting_artifact": {
            "id": artifact_info["id"],
            "snapshotId": artifact_info["snapshot"],
        }
    },
)

In [ ]:
job2_info = client.jobs.get_by_id(job_id=job2_response["id"])
pprint(job2_info)